In [38]:
import os
import openai
from dotenv import load_dotenv, find_dotenv

In [39]:
# Load environment variables (OpenAI API Key)
_ = load_dotenv(find_dotenv())
openai.api_key = os.environ['OPENAI_API_KEY']

In [40]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.schema.runnable import RunnableMap

In [42]:
# Step 1: Vector Store Creation
# Create a vector store from given texts
vectorstore = DocArrayInMemorySearch.from_texts(
    ["harrison worked at kensho", "bears like to eat honey"],
    embedding=OpenAIEmbeddings()
)

# Convert the vector store into a retriever
retriever = vectorstore.as_retriever()


In [43]:
# Step 2: Prompt Template
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

In [44]:
# Step 3: Model Definition
# Define the OpenAI chat model
model = ChatOpenAI(model_name="gpt-3.5-turbo")

In [45]:
# Define an output parser to format the output
output_parser = StrOutputParser()

In [46]:
# Step 4: LCEL Chain Definition
# Define a RunnableMap that processes input to extract context, format the prompt, and generate an answer
chain = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
}) | prompt | model | output_parser

In [48]:
# Step 5: Testing the Chain
# Example questions for testing
question_1 = {"question": "where did harrison work?"}
question_2 = {"question": "what do bears like to eat?"}

# Invoke the chain for each question
response_1 = chain.invoke(question_1)
response_2 = chain.invoke(question_2)

# Print the outputs
print(f"Response 1: {response_1}")
print(f"Response 2: {response_2}")

Response 1: Harrison worked at Kensho.
Response 2: Bears like to eat honey.


In [49]:
# Step 6: Inputs Handling (Optional, Additional Example)
inputs = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
})

# Test with inputs.invoke
response_from_inputs = inputs.invoke({"question": "where did harrison work?"})
print(f"Response from inputs.invoke: {response_from_inputs}")

Response from inputs.invoke: {'context': [Document(page_content='harrison worked at kensho'), Document(page_content='bears like to eat honey')], 'question': 'where did harrison work?'}
